In [8]:
url_sesson_list = list["2021-2022-apertura", 
                       "2021-2022-clausura"
                       "2021-2022-clausura-playoffs"]
print(url_sesson_list[0])
url = "https://www.livefutbol.com/todos_partidos/mex-primera-division-" + url_sesson_list[0] + "/"  #2021-2022-apertura/"

TypeError: There are no type variables left in list['2021-2022-apertura', '2021-2022-clausura2021-2022-clausura-playoffs']

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Puedes cambiar este por el link de cualquier otra liga siempre y cuando se de la ruta /todos_partidos
url_sesson_list = list("2021-2022-apertura", 
                       "2021-2022-clausura"
                       "2021-2022-clausura-playoffs")
url = "https://www.livefutbol.com/todos_partidos/mex-primera-division-" + url_sesson_list[0] + "/"  #2021-2022-apertura/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

select_element = soup.find('select', {'name': 'saison'})

urls = ['https://www.livefutbol.com' + option['value'] for option in select_element.find_all('option')]

matches_df =[]
for url in urls:
    nombre = url.split('-')[-1].strip('/')
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    matches = []

    for row in soup.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) > 5:  
            match = {
                "date": cols[0].get_text(strip=True),
                "time": cols[1].get_text(strip=True),
                "torneo": nombre,
                "team_home": cols[2].get_text(strip=True),
                "team_away": cols[4].get_text(strip=True),
                "score": cols[5].get_text(strip=True)
            }
            matches.append(match)
            
    matches = pd.DataFrame.from_dict(matches, orient = 'columns')
    matches = matches[matches['score']!='-:-']
    
    for i, row in matches.iterrows():
        try:
            final, halftime = row['score'].strip().split(" ")
            final_home, final_away = final.split(':')
            ht_home, ht_away = halftime.strip('()').split(':')

            matches.loc[i, 'final_score_home'] = int(final_home)
            matches.loc[i, 'final_score_away'] = int(final_away)
            matches.loc[i, 'halftime_score_home'] = int(ht_home)
            matches.loc[i, 'halftime_score_away'] = int(ht_away)
        except Exception as e:
            try:
                final = row['score'].strip()
                final_home, final_away = final.split(':')
                matches.loc[i, 'final_score_home'] = int(final_home)
                matches.loc[i, 'final_score_away'] = int(final_away)
            except:   
                matches.loc[i, 'final_score_home'] = None
                matches.loc[i, 'final_score_away'] = None
                matches.loc[i, 'halftime_score_home'] = None
                matches.loc[i, 'halftime_score_away'] = None
        
    matches_df.append(matches)
matches_df = pd.concat(matches_df)

TypeError: list expected at most 1 argument, got 2

In [12]:
matches_df.to_csv("temp2022-2.csv")